### IMPORT LIBRARY

In [1]:
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.file import File
from office365.runtime.auth.client_credential import ClientCredential
from office365.runtime.client_request_exception import ClientRequestException
import datetime
import pytz
import pandas as pd
import numpy as np
import csv
import glob
import pyodbc
import os 
import json
from io import BytesIO
import io
import platform
from function.PyToSp import *
from tqdm import tqdm
from urllib.parse import quote_plus
import numpy as np
import pandas as pd
from sqlalchemy import create_engine, event
import pyodbc
import requests
import inspect
from validateSamSung import *
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from urllib.parse import quote_plus
import msal
from itertools import chain
from send_email import *

### CONNECT TO AZURE SQL

In [2]:
import json
f = open ('database_information.json', "r")
qq = json.loads(f.read())
f.close()
ini_cnt_str ='Driver={driver_str};Server=tcp:hkazdevsqld3vnreserch.database.windows.net,1433;database={database};Uid={username};Pwd={password};Encrypt=yes;Authentication=ActiveDirectoryPassword;Connection Timeout=30;'.format(**qq)
quoted = quote_plus(ini_cnt_str)
cnt_str = 'mssql+pyodbc:///?odbc_connect={}'.format(quoted)
engine = create_engine(cnt_str)

### CONNECT TO SHAREPOINT

In [3]:
header_BIHub = 'share_point_BIHub'
config_BIHub = read_config_json(config_path, header_BIHub)
BIHub = SharePoint(config_BIHub)
BIHub.check_connect()

Web site title: S22M Research & BI Hub _ Successful Connection!


In [4]:
#Tất cả các tỉnh
relative_url = "/sites/BIHub/Shared Documents/Advisory Data/Project/SAMSUNG/TEST"
sp_object = relative_url.split('/')[2].replace('-','')
list_folder = eval(sp_object).get_content_url(relative_url,return_list_folder=True)

Folder name: /sites/BIHub/Shared Documents/Advisory Data/Project/SAMSUNG/TEST/2023


In [5]:
#lấy năm
list_folder_sub1=[]
for i in list_folder:
    ls=eval(sp_object).get_content_url(i,return_list_folder=True)
    list_folder_sub1=list_folder_sub1+ls

Folder name: /sites/BIHub/Shared Documents/Advisory Data/Project/SAMSUNG/TEST/2023/Q2


In [6]:
quarter = ['Q2']#Chọn năm cần import
year = ['2023']#Chọn quý cần import
#-------------------------------------------------------
df_summ_file = pd.DataFrame({'Name':[],'ServerRelativeUrl':[], 'TimeLastModified':[], 'ModTime':[], 'Modified_by_ID':[]})
for i in list_folder_sub1:
    if i.split('/')[-2] in year and i.split('/')[-1] in quarter:
        df_summ_file = pd.concat([df_summ_file, eval(sp_object).get_content_url(i)])
list_file = df_summ_file['ServerRelativeUrl'].to_list()
#History file
df_query=pd.DataFrame(df_summ_file).reset_index(drop=True)
df_summ_file = df_summ_file.reset_index(drop=True)

Folder name: 
Files name: /sites/BIHub/Shared Documents/Advisory Data/Project/SAMSUNG/TEST/2023/Q2/SAMSUNG_HCM_20230630.xlsx


In [7]:
url = []
for i in list_file:
    data_key = i.split('/')[-1].split('.')[0].split('_')[-1]
    if str(data_key) not in ('x'):
        url.append(i)
url        

['/sites/BIHub/Shared Documents/Advisory Data/Project/SAMSUNG/TEST/2023/Q2/SAMSUNG_HCM_20230630.xlsx']

## Import

In [8]:
#Create empty df for checking dictionary
df_dict = pd.DataFrame(columns=['File_Name', 'Missing_Values', 'Flag'])
df_temp_flat_fdi = pd.DataFrame()
df_flat_fdi = pd.DataFrame()
#-------------------------------------------------------
'''Get data''' 
for file_url in tqdm(url):
#     choose_sheetname = int(input('Choose 1. Retailer Input or 2. Street Retail: '))
#     if choose_sheetname == 1:
#         sheetname ='Retailer Input'
#     elif choose_sheetname == 2:
#         sheetname ='Street Retail'
#     else:
#         print('Invalid choice. Please choose 1 or 2.')
#         continue
    data, file_name, sector = get_data(relative_url, file_url)
    #data, file_name, sector = get_data(relative_url, file_url, sheetname)
    
    #-------------------------------------------------------
    data = check_date_key(file_url, data)#Check format date_key in flat file
    #-------------------------------------------------------
#     '''Validation step'''
#     #Remove unfortmated values
#     #data = remove_unformated_character(type_sector = 2, sector = sector, file_name = file_name, data = data)
#     data = remove_unformated_character(data)
#     #Remove unicode characters
#     for i in columns_that_need_unidecode:
#         data[i] = remove_unicode(data[i])
    #Check dictionary
    lst_dict = ['City', 'District', 'Ward']
    lst_cls = ['Province', 'Address_District', 'Address_Ward']
    for i, j in zip(lst_cls, lst_dict):
        data, df_dict = check_dictionary(df_dict, file_name, data, i, j, sector, engine, sp_object)
    print(df_dict)
    if len(df_dict) == 0:
        print(colored('Validate succesfully','green'))
        data = Generate_Additional_Columns(data,df_summ_file,BIHub,engine,file_url)
        df_temp_flat_fdi = pd.concat([df_temp_flat_fdi, data], axis=0)
        df_flat_fdi = tracking_flat_file(df_temp_flat_fdi, sector)
        print(data)
        insert_to_fresh(file_url, data, cnt_str)
    else:
        pass


  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

Empty DataFrame
Columns: [File_Name, Missing_Values, Flag]
Index: []
Validate succesfully
Unknown sector
    Date_Key          Province    Retailer_Group       Retailer_Name  \
0   20230630  Ho Chi Minh City  The Gioi Di Dong  The Gioi Di Dong 1   
1   20230630  Ho Chi Minh City  The Gioi Di Dong  The Gioi Di Dong 2   
2   20230630  Ho Chi Minh City  The Gioi Di Dong  The Gioi Di Dong 3   
3   20230630  Ho Chi Minh City  The Gioi Di Dong  The Gioi Di Dong 4   
4   20230630  Ho Chi Minh City  The Gioi Di Dong  The Gioi Di Dong 5   
5   20230630  Ho Chi Minh City  The Gioi Di Dong  The Gioi Di Dong 6   
6   20230630  Ho Chi Minh City     Dien May Xanh     Dien May Xanh 1   
7   20230630  Ho Chi Minh City     Dien May Xanh     Dien May Xanh 2   
8   20230630  Ho Chi Minh City     Dien May Xanh     Dien May Xanh 3   
9   20230630  Ho Chi Minh City     Dien May Xanh     Dien May Xanh 4   
10  20230630  Ho Chi Minh City     Dien May Xanh     Dien May Xanh 5   
11  20230630  Ho Chi Minh City 

  0%|                                                                                            | 0/1 [00:13<?, ?it/s]


DataError: (pyodbc.DataError) ('22018', '[22018] [Microsoft][ODBC Driver 17 for SQL Server]Invalid character value for cast specification (0) (SQLExecute)')
[SQL: INSERT INTO [Fresh].[SAMSUNG_Retailer_Input] ([Date_Key], [Province], [Retailer_Group], [Retailer_Name], [Latitude], [Longtitude], [Address_No], [Address_Street], [Address_District], [Address_Ward], [Mobile], [Electronics], [Mix_Type], [Land_Area], [Gross_Floor_Area], [No_Of_Level], [Online_Share_On_Revenue], [Offline_Share_On_Revenue], [File_Date], [Import_Date], [File_Name]) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)]
[parameters: ((20230630, 'Ho Chi Minh City', 'The Gioi Di Dong', 'The Gioi Di Dong 1', 10.790159, 106.729958, 198, 'Tran Nao', 'District 2', 'Binh An Ward', None, None, None, 200, 200.0, 1, None, None, datetime.datetime(2023, 7, 20, 6, 44, 10), '2023-07-20 13:44:39.017', 'SAMSUNG_HCM_20230630'), (20230630, 'Ho Chi Minh City', 'The Gioi Di Dong', 'The Gioi Di Dong 2', 10.845885, 106.